# Naive Bayes Gaussian (Cardiovascular Disease Prediction)

## Explanation

### Objective

created a machine learning model that can predict whether a person has cardiovascular disease or not, using the Naive Bayes algorithm.

### Column Descriptions

Age | Objective Feature | age | int (days)

Height | Objective Feature | height | int (cm) |

Weight | Objective Feature | weight | float (kg) |

Gender | Objective Feature | gender | categorical code |

Systolic blood pressure | Examination Feature | ap_hi | int |

Diastolic blood pressure | Examination Feature | ap_lo | int |

Cholesterol | Examination Feature | cholesterol | 1: normal, 2: above normal, 3: well above normal |

Glucose | Examination Feature | gluc | 1: normal, 2: above normal, 3: well above normal |

Smoking | Subjective Feature | smoke | binary |

Alcohol intake | Subjective Feature | alco | binary |

Physical activity | Subjective Feature | active | binary |

Presence or absence of cardiovascular disease | Target Variable | cardio | binary |

## A. Data Preparation

## B. Data Preprocessing

## C. Exploratory Data Analysis

## D. Modelling

## E. Evaluation